# **Custom Lab Experience**

## Imports

In [ ]:
# Imports

import glob
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import re as Read

## Raw Data

In [ ]:
# Import CSVs

raw_data = {}

# Pulls AoA out of file name regardless of how it was named
def extract_number(text):
    match = Read.search(r'\d+', text)
    if match:
        return int(match.group())

# Saves every DF in the raw_data dictionary by Reynolds --> Report --> AoA
for re in ['250k', '320k', '500k', '750k']:
    directory = 'Reports ' + re
    raw_data[f'{re}'] = {}
    for report in ['Drag', 'Drag Coefficient', 'Lift', 'Lift Coefficient', 'LoD', 'Residuals']:
        raw_data[f'{re}'][f'{report}'] = {}
        for file in glob.glob(os.path.join(f'{directory}\{report}', '*.csv')):
            data = pd.read_csv(file)

            # Pull AoA
            split = file.split()
            for s in split:
                if (s[0].isnumeric()) and ('k' not in s):
                    num = extract_number(s)

            raw_data[f'{re}'][f'{report}'][f'{num}'] = data

## Composite Data

In [ ]:
# Composite Data

result_dataframes = {}

# Iterate through the raw_data dictionary
for re_value, re_data in raw_data.items():
    for report, report_data in re_data.items():
        # Initialize lists for nums and vals
        nums = []
        vals = []
        
        for num, data in report_data.items():
            # Extract the final iteration of each AoA case
            last_row = data.iloc[-1, 1]
            vals.append(last_row)
            nums.append(num)
        
        # Composite DF
        df_name = f'{re_value} {report}'
        result_df = pd.DataFrame({'Value': vals}, index=nums)
                # Convert the index to numeric and sort
        result_df.index = pd.to_numeric(result_df.index)
        result_df.sort_index(inplace=True)
        
        # Store all DFs in central location
        if df_name not in result_dataframes:
            result_dataframes[df_name] = result_df
        else:
            result_dataframes[df_name] = result_dataframes[df_name].append(result_df)

## Plotting

In [ ]:
# Individual Plots

for df_name, df in result_dataframes.items():
    # Determine the y-axis label based on the report type
    if 'Drag' in df_name:
        if 'Coefficient' in df_name:
            y_label = 'Drag Coefficient'
        else:
            y_label = 'Drag (N)'
    elif 'Lift' in df_name:
        if 'Coefficient' in df_name:
            y_label = 'Lift Coefficient'
        else:
            y_label = 'Lift (N)'
    else:
        y_label = df_name.split()[-1]

    # Plot each DataFrame
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df['Value'], marker='o', linestyle='-', label=df_name)
    plt.title(f'{df_name} vs AoA')
    plt.xlabel('AoA (Deg)')
    plt.ylabel(y_label)
    plt.grid(True)

    # Save the plot without displaying it
    plt.savefig(f'Plots/Individual/{df_name}_vs_AoA.png', bbox_inches='tight')  # Use bbox_inches to ensure tight layout

    # Close the current figure to avoid potential conflicts with future plots
    plt.close()




In [ ]:
# Shared Plots

# Create a dictionary to store DataFrames by plot type
plot_type_dataframes = {}

# Iterate through the result_dataframes dictionary
for df_name, df in result_dataframes.items():
    # Extract the plot type from the DataFrame name
    plot_type = ' '.join(df_name.split()[1:])

    # Store DataFrames by plot type
    if plot_type not in plot_type_dataframes:
        plot_type_dataframes[plot_type] = [(df, df_name.split()[0])]
    else:
        plot_type_dataframes[plot_type].append((df, df_name.split()[0]))

# Create separate plots for each plot type
for plot_type, dfs in plot_type_dataframes.items():
    plt.figure(figsize=(10, 6))
    for df, label in dfs:
        plt.plot(df.index, df['Value'], marker='o', linestyle='-', label=label)  # Use label for legend

    plt.title(f'{plot_type} vs Re')
    plt.xlabel('AoA (Deg)')
    
    if plot_type == 'Lift' or plot_type == 'Drag':
        plt.ylabel(f'{plot_type} (N)')
    else:
        plt.ylabel(plot_type)

    plt.legend()
    plt.grid(True)
    
    # Save the plot without displaying it
    plt.savefig(f'Plots/Shared/{plot_type}_vs_Re.png', bbox_inches='tight')  # Use bbox_inches to ensure tight layout

    # Close the current figure to avoid potential conflicts with future plots
    plt.close()


## Old Versions

In [ ]:
dragcsvdir = 'C:\\Users\\Harrison\\OneDrive - University of Cincinnati\\Year 5\\Aero Lab\\Lab Work\\Custom Lab\\320k Reynolds - Stoelting\\Reports\\Drag'
Cdcsvdir = 'C:\\Users\\Harrison\\OneDrive - University of Cincinnati\\Year 5\\Aero Lab\\Lab Work\\Custom Lab\\320k Reynolds - Stoelting\\Reports\\Drag Coefficient'
liftcsvdir = 'C:\\Users\\Harrison\\OneDrive - University of Cincinnati\\Year 5\\Aero Lab\\Lab Work\\Custom Lab\\320k Reynolds - Stoelting\\Reports\\Lift'
Clcsvdir = 'C:\\Users\\Harrison\\OneDrive - University of Cincinnati\\Year 5\\Aero Lab\\Lab Work\\Custom Lab\\320k Reynolds - Stoelting\\Reports\\Lift Coefficient'
LoDcsvdir = 'C:\\Users\\Harrison\\OneDrive - University of Cincinnati\\Year 5\\Aero Lab\\Lab Work\\Custom Lab\\320k Reynolds - Stoelting\\Reports\\LoD'

# get all the csv files in that directory (assuming they have the extension .csv)
dragcsvfiles = glob.glob(os.path.join(dragcsvdir, '*.csv'))
Cdcsvfiles = glob.glob(os.path.join(Cdcsvdir, '*.csv'))
liftcsvfiles = glob.glob(os.path.join(liftcsvdir, '*.csv'))
Clcsvfiles = glob.glob(os.path.join(Clcsvdir, '*.csv'))
LoDcsvfiles = glob.glob(os.path.join(LoDcsvdir, '*.csv'))

# loop through the files and read them in with pandas
dragdataframes = []  # a list to hold all the individual pandas DataFrames
Cddataframes = []
Liftdataframes = []
Cldataframes = []
LoDdataframes = []

#using individual for loops so it is easier to comment out whichever set of files is not needed
for csvfile in dragcsvfiles:
    df = pd.read_csv(csvfile)
    dragdataframes.append(df)

print("Drag Complete")

for csvfile in Cdcsvfiles:
    df = pd.read_csv(csvfile)
    Cddataframes.append(df)

print("Cd Complete")

for csvfile in liftcsvfiles:
    df = pd.read_csv(csvfile)
    Liftdataframes.append(df)

print("Lift Complete")

for csvfile in Clcsvfiles:
    df = pd.read_csv(csvfile)
    Cldataframes.append(df)

print("Cl Complete")

for csvfile in LoDcsvfiles:
    df = pd.read_csv(csvfile)
    LoDdataframes.append(df)

print("LoD Complete")

### Breaking into arrays with relevant data

In [ ]:
#Creating arrays that have number of rows = # of files and columns = # of rows in csv
Drags = np.zeros((len(dragdataframes), len(dragdataframes[0])))
CDs = np.zeros((len(Cddataframes), len(Cddataframes[0])))
Lifts = np.zeros((len(Liftdataframes), len(Liftdataframes[0])))
CLs = np.zeros((len(Cldataframes), len(Cldataframes[0])))
LoDs = np.zeros((len(LoDdataframes), len(LoDdataframes[0])))

#populating the arrays with the useful data
#Drag
for i in range(len(dragdataframes)):
    temp_frame = dragdataframes[i]
    for j in range(len(dragdataframes[0])):
        Drags[i,j] = temp_frame.iloc[j,1]

#CD
for i in range(len(Cddataframes)):
    temp_frame = Cddataframes[i]
    for j in range(len(Cddataframes[0])):
        CDs[i,j] = temp_frame.iloc[j,1]

#Lift
for i in range(len(Liftdataframes)):
    temp_frame = Liftdataframes[i]
    for j in range(len(Liftdataframes[0])):
        Lifts[i,j] = temp_frame.iloc[j,1]

#CL
for i in range(len(Cldataframes)):
    temp_frame = Cldataframes[i]
    for j in range(len(Cldataframes[0])):
        CLs[i,j] = temp_frame.iloc[j,1]

#LoD
for i in range(len(LoDdataframes)):
    temp_frame = LoDdataframes[i]
    for j in range(len(LoDdataframes[0])):
        LoDs[i,j] = temp_frame.iloc[j,1]


### Averaging the values after the initial start (after the 700th iteration)

In [ ]:
#creating a pandas frame to store the averaged values
test_num = len(dragdataframes) #number of tests ran based off the number of files
init_data = {'Drag (N)': [0], 'C_D': [0], 'Lift (N)': [0], "C_L": [0], "L/D": [0]}
avg_df = pd.DataFrame(init_data)

#Finding the average for each AoA (row) of the column
for j in range(len(avg_df)):
    for i in range(test_num):
        avg = np.average(Drags[i,700:])
        avg_df.at[i, "Drag (N)"] = avg

        avg = np.average(CDs[i,700:])
        avg_df.at[i, "C_D"] = avg

        avg = np.average(Lifts[i,700:])
        avg_df.at[i, "Lift (N)"] = avg

        avg = np.average(CLs[i,700:])
        avg_df.at[i, "C_L"] = avg

        avg = np.average(LoDs[i,700:])
        avg_df.at[i, "L/D"] = avg

avg_df